Vardas, pavardė: Mėta Bambalaitė

LSP: 1813061

Pirma užduotis, variantas A = 2, B = 5, C = 4

*   Objektų modelis: SSD
*   Papildoma nuostolių funkcija: Weighted IoU (pakeista į DR Loss)
*   Duomenų klasės: Headphones, Coffee, Raven


Mount to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Access google drive folder

In [ ]:
%cd /content/gdrive/MyDrive/GMM_first_project/pytorch-ssd/

/content/gdrive/MyDrive/GMM_first_project/pytorch-ssd


In [ ]:
ls

data/                 LICENSE                    run_ssd_example_output.jpg
draw_eval_results.py  models/                    run_ssd_example.py
eval_results/         open_images_downloader.py  train_ssd.py
eval_ssd.py           raven_test.jpg             vision/
gun_test.jpg          README.md


Clone model repository to the folder

In [ ]:
!git clone https://github.com/qfgaohao/pytorch-ssd/

Cloning into 'pytorch-ssd'...
remote: Enumerating objects: 812, done.
remote: Total 812 (delta 0), reused 0 (delta 0), pack-reused 812
Receiving objects: 100% (812/812), 1.05 MiB | 6.18 MiB/s, done.
Resolving deltas: 100% (544/544), done.


Install libraries for downloading images

In [ ]:
!pip install openimages

Install Boto3 to be able to use OpenImages

In [ ]:
pip install boto3

Download OpenImages classes "Headphones,Coffee,Raven"

In [ ]:
!python open_images_downloader.py --root data/open_images/ --class_names "Headphones,Coffee,Raven" --num_workers 20

Retrain pretrained MobilenetV1 SSD model

In [ ]:
!python train_ssd.py --dataset_type open_images --datasets data/open_images --net mb1-ssd --pretrained_ssd models/mobilenet-v1-ssd-mp-0_675.pth --scheduler cosine --lr 0.01 --t_max 100 --validation_epochs 15 --num_epochs 15 --base_net_lr 0.001  --batch_size 5

2021-04-27 14:31:27,109 - root - INFO - Use Cuda.
2021-04-27 14:31:27,109 - root - INFO - Namespace(balance_data=False, base_net=None, base_net_lr=0.001, batch_size=5, checkpoint_folder='models/', dataset_type='open_images', datasets=['data/open_images'], debug_steps=100, extra_layers_lr=None, gamma=0.1, lr=0.01, milestones='80,100', momentum=0.9, net='mb1-ssd', num_epochs=15, num_workers=4, pretrained_ssd='models/mobilenet-v1-ssd-mp-0_675.pth', resume=None, scheduler='cosine', t_max=100.0, use_cuda=True, validation_dataset=None, validation_epochs=15, weight_decay=0.0005)
2021-04-27 14:31:27,151 - root - INFO - Prepare training datasets.
2021-04-27 14:31:32,199 - root - INFO - Dataset Summary:Number of Images: 3351
Minimum Number of Images for a Class: -1
Label Distribution:
	Coffee: 2384
	Headphones: 1255
	Raven: 567
2021-04-27 14:31:32,203 - root - INFO - Stored labels into file models/open-images-model-labels.txt.
2021-04-27 14:31:32,204 - root - INFO - Train dataset size: 3351
/usr

To test use:

In [19]:
from vision.ssd.mobilenetv1_ssd import create_mobilenetv1_ssd, create_mobilenetv1_ssd_predictor
from vision.utils.misc import Timer
import cv2
import sys
from google.colab.patches import cv2_imshow


net_type = 'mb1-ssd'
#for focal loss+classification_loss use 'models/mb1-ssd-Epoch-14-Loss-2.3152370490525898.pth'
#for focal loss+dr loss use 'models/mb1-ssd-Epoch-14-Loss-1.5965070580181322.pth'
model_path = 'models/mb1-ssd-Epoch-14-Loss-2.3152370490525898.pth'
label_path = 'models/open-images-model-labels.txt'
image_paths = 'test_img/raven_test.jpg test_img/coffee_test.jpg test_img/headphones_test.jpg test_img/hp_c_test.jpg'
split_image_paths = image_paths.split(' ')

class_names = [name.strip() for name in open(label_path).readlines()]

net = create_mobilenetv1_ssd(len(class_names), is_test=True)

net.load(model_path)

predictor = create_mobilenetv1_ssd_predictor(net, candidate_size=200)
for image_path in split_image_paths:
  orig_image = cv2.imread(image_path)
  image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
  boxes, labels, probs = predictor.predict(image, 10, 0.4) # 0.4 with classification_loss, 0.9 with dr_loss

  for i in range(boxes.size(0)):
      box = boxes[i, :]
      cv2.rectangle(orig_image, (box[0], box[1]), (box[2], box[3]), (255, 255, 0), 4)
      label = f"{class_names[labels[i]]}: {probs[i]:.2f}"
      cv2.putText(orig_image, label,
                  (box[0] + 20, box[1] + 40),
                  cv2.FONT_HERSHEY_SIMPLEX,
                  1,  # font scale
                  (255, 0, 255),
                  2)  # line type
  cv2_imshow(orig_image)

KeyboardInterrupt: ignored

Deploy via REST API

Install required Dependencies

In [5]:
!pip install Flask==1.0.3
!pip install flask-ngrok

     |████████████████████████████████| 92kB 3.6MB/s 
  Found existing installation: Flask 1.1.2
    Uninstalling Flask-1.1.2:
      Successfully uninstalled Flask-1.1.2


Run on ngrok server

Test on POSTMAN

Choose GET method

Paste provided ngrok url with /predict at the end, for example: http://236e3f00f68d.ngrok.io/predict

Insert params:

*   image_path - test_img/raven_test.jpg

use test_img/hp_c_test.jpg to test multiple predicted boxes
*   model_path - models/mb1-ssd-Epoch-14-Loss-2.3152370490525898.pth
*   label_path - models/open-images-model-labels.txt







In [39]:
!python rest_app.py

 * Serving Flask app "rest_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
